# Title

Project website: https://docs.google.com/document/d/1GkK28wOyjTPZEZuOumKPU0z1NzVT_9sxETjPOcLPVYo/edit?tab=t.0#heading=h.qifoo7co6qtd

Professor website: https://malchiodi.di.unimi.it/teaching/AMD-DSE/2024-25/en

In [2]:
!git clone https://github.com/chiesastefano/massiveData
import os
os.chdir("massiveData")

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [3]:
# Update alternatives to set Python 3.10 as the default
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1

In [5]:
import subprocess
import sys
# Install required packages from the requirement.txt file if not already installed
!pip install -r requirements.txt trigger a runtime restart
exit()

FileExistsError: [Errno 17] File exists: '/usr/bin/python3.10' -> '/usr/bin/python'

In [ ]:
import os
import subprocess
import sys

# Set environment variables for Kaggle API
os.environ['KAGGLE_USERNAME'] = "ilchurch"
os.environ['KAGGLE_KEY'] = "5449548f4b6c8556c8f48f4a31b4ea6e"

# Ensure kaggle is installed
try:
    import kaggle
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "kaggle"])
    import kaggle

# Import and authenticate using KaggleApi
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

# Check if the dataset file already exists
if not os.path.exists('data/Books_rating.csv'):
    api.dataset_download_files('mohamedbakhet/amazon-books-reviews', path='data', unzip=True)
else:
    print("Dataset already exists.")

In [ ]:
# Load the dataset with spark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("JaccardSimilarity") \
    .master("local[*]") \
    .getOrCreate()
books_rating = spark.read.csv("data/Books_rating.csv", header=True, inferSchema=True)

In [ ]:
# Display the first few rows of the books_rating dataset
books_rating.show(5)

# Create a subsample of the dataset for performance reasons
I will use a sample of the dataset as a test case. My local and Google Collab computers are not powerful enough to handle the full dataset. I will use *sample* as a parameter can it can be changed to the desired sample size.

In [ ]:
sample = 0.01 # 1% of the dataset

In [ ]:
books_rating_sample = books_rating.sample(fraction=sample, seed=42)
books_rating_sample.show(5)
sample_size = books_rating_sample.count()

In [ ]:
sample_size

# Checking for null values in the 'review/text' column
Since we are using text data, we need to check for null values in the 'review/text' column in order to take a decision on how to handle them.

In [ ]:
# Check for null values in the 'review/text' column
null_rows_df = books_rating.filter(
    books_rating['review/text'].isNull() | (books_rating['review/text'].rlike('^\\s*$'))
)

null_rows_df.show(30)

Since the 'review/text' null values are more or less duplicates.

They share the same columns, besides the identifiers and the Title. Although the Title is not the same, with a qualitative check we can see they are about the same book.

In [ ]:
from pyspark.sql.functions import col
filtered_df = books_rating.filter(
    col("Title").contains("Lord of the Rings")
).select("Title", "review/text").limit(100)

# Show the result
filtered_df.show(100, truncate=False)

There are many others Lord of The Rings reviews. I think we can't discard the NA values and it's better to keep them. We can use also the Title as similarity check.

In [ ]:
# Check for null values in the 'review/text' column
null_rows_df_sample = books_rating_sample.filter(
    books_rating_sample['review/text'].isNull() | (books_rating_sample['review/text'].rlike('^\\s*$'))
)

null_rows_df_sample.show(30)

The is a null value in the subsample

# Jaccard Similarity

## Tokenization
Since the dataset is too big, I will use a subsample of the dataset to test the Jaccard Similarity approach, for test purposes. I will then try top scale it to the full dataset.
In addition, I will use the MapReduce approach to parallelize the computation of the Jaccard Similarity.

In [ ]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, StringType

In [ ]:
tokenizer = RegexTokenizer(inputCol="review/text", outputCol="reviews/tokens", pattern="\\W") # \\W is a regex pattern that matches any non-word character

It tokenizes the text into words. I am trying this approach because I believe it will be faster then some neural network based approach, since this use sparks. It will have some limitations, since it does not take in account of compound words, but I think it will be good for the first approach.

In [ ]:
stopwords_remover = StopWordsRemover(inputCol="reviews/tokens", outputCol="tokens/nostopwords")

StopWordsRemover() uses a predefined list of stop words in English. It removes common words that do not carry much meaning, such as "the", "is", "in", etc. This is important, since the Jaccard Similarity is based on the number of common words between two texts. If we don't remove the stop words, the Jaccard Similarity will be biased by the number of stop words in the texts, which are not relevant for the meaning of the texts.

In [ ]:
def remove_numbers(tokens):
    return [token for token in tokens if not token.isdigit()]

Numbers are often not relevant for the meaning of a review. In our dataset, the score is stored in a separated column, so we can remove the numbers from the text.

In [ ]:
remove_numbers_udf = udf(remove_numbers, ArrayType(StringType())) # Specify the return type as ArrayType(StringType), meaning an array/list of strings

Sparks does not work as a Python object, so we need UDF (user defined function) to use it in the pipeline.

In [ ]:
tokenized_books_rating_sample = tokenizer.transform(books_rating_sample) # Apply the tokenizer to the dataset
tokenized_reviews_nostopwords_sample = stopwords_remover.transform(tokenized_books_rating_sample) # Remove the stop words from the tokenized dataset
tokenized_rw_nsw_nn_sample = tokenized_reviews_nostopwords_sample.withColumn("tokens/nostopwords/nonumbers", remove_numbers_udf(col("tokens/nostopwords"))) # Remove the numbers from the dataset

In [ ]:
import sys
print("Python Version:", sys.version)  # Check if it prints Python 3.12 in both worker and driver.

In [ ]:
# Verify that the environment variable is set correctly

books_rating_sample.show(10)
tokenized_rw_nsw_nn_sample.show(10)

In [ ]:
from pyspark.ml.feature import CountVectorizer
from pyspark.sql.functions import col

df_safe = tokenized_rw_nsw_nn_sample \
    .filter(col("review/text").isNotNull())

cv = CountVectorizer(
    inputCol="tokens/nostopwords/nonumbers",  # No renaming here
    outputCol="features",
    vocabSize=5000,
    minDF=10
)

model = cv.fit(df_safe)
vocab = model.vocabulary


In [ ]:
top30 = model.vocabulary[:30]
print("Top 30 tokens (approx. by frequency):")
for i, token in enumerate(top30, start=1):
    print(f"{i:2d}. {token}")

This method approximates the top 30 words in the vocaboulary. However, it doesn't show the frequency, since CountVectorizer() only approximates the real frequency of the top words. I want to find them.

In [ ]:
from pyspark.sql.functions import explode, col

# 1) Explode the original tokens column into one token per row
exploded = df_safe.select(explode(col("tokens/nostopwords/nonumbers")).alias("token"))

# 2) Filter to only the top-K tokens
topK_set = set(model.vocabulary[:20])  # assumes you want top 20 from vocab
filtered = exploded.filter(col("token").isin(topK_set))

# 3) Group and count
exact_counts = (
    filtered
      .groupBy("token")
      .count()
      .orderBy(col("count").desc())
)

exact_counts.show(truncate=False)

In [ ]:
threshold = 0.20 * sample_size
print(threshold)

I defined a function that only keeps the tokens which are not in the custom_stopwords list

In [ ]:
high_freq_tokens = (
    exact_counts
      .filter(col("count") > threshold)
      .select("token")
      .rdd.flatMap(lambda x: x)
      .collect()
)

In [ ]:
high_freq_tokens

In [ ]:
# Define a UDF to filter them out
def remove_custom_stopwords(tokens):
    return [t for t in tokens if t not in high_freq_tokens]

remove_udf = udf(remove_custom_stopwords, ArrayType(StringType()))

tokenized_sample_2 = tokenized_rw_nsw_nn_sample.withColumn(
    "tokens_clean",
    remove_udf(col("tokens/nostopwords/nonumbers"))
)

In [ ]:
df_safe = tokenized_sample_2.filter(col("review/text").isNotNull())

cv = CountVectorizer(
    inputCol="tokens_clean",
    outputCol="features",
    vocabSize=5000,
    minDF=10
)

model2 = cv.fit(df_safe)
vocab2 = model2.vocabulary


In [ ]:
top30 = model2.vocabulary[:30]
print("Top 30 tokens (approx. by frequency):")
for i, token in enumerate(top30, start=1):
    print(f"{i:2d}. {token}")

## Local Sensitive Hashing

Since the project must be scalable, we can't compute the Jaccard similarity on the whole dataset. Instead, we will


In [ ]:
sim_threshold = 0.6

In [ ]:
tokenized_featurized = model2.transform(df_safe)

The Hash Function doesn't work with strings, it need vectors

In [ ]:
tokenized_featurized.show(10)

In [ ]:
from pyspark.ml.feature import MinHashLSH
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import when

mh = MinHashLSH(
    inputCol="features",
    outputCol="hashes",
    numHashTables=10
)
mhModel = mh.fit(tokenized_featurized)
hashed = mhModel.transform(tokenized_featurized)

Map tokens to a digit and select the min among the hash functions

In [ ]:
candidates = mhModel.approxSimilarityJoin(
    hashed, hashed,
    threshold=1.0 - sim_threshold,
    distCol="jaccardDist"
).filter(col("datasetA.Id") < col("datasetB.Id")) # Filters out redundant pairs

results_pandas = candidates.toPandas()

Self join and then compute the Jaccard Similarity for each option and filters out pairs under the threshold.

In [ ]:
results = candidates.select(
        col("datasetA.Id").alias("idA"),
        col("datasetB.Id").alias("idB"),
        (1.0 - col("jaccardDist")).alias("approxJaccard")
    )